In [2]:
import tensorflow as tf
import warnings
import sys
if not sys.warnoptions:
    warnings.simplefilter("ignore")
import numpy as np
from tqdm import tqdm
import pandas as pd
import re
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec 
from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import TaggedDocument
import multiprocessing
from sklearn import utils
from nltk.corpus import stopwords
from sklearn.cross_validation import train_test_split
SEED = 2000


train=pd.read_csv('train_E6oV3lV.csv')
test=pd.read_csv('test_tweets_anuFYb8.csv')

target=train.label
train=train.drop('label',1)
data=train.append(test)
tweet =data.tweet


# pre-processing
print("Text Preprocessing ====>")
data['tweet']=data['tweet'].apply(lambda x: x.lower())
data['tweet']=data['tweet'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
data['tweet']=data['tweet'].apply(lambda x: x.replace('user',''))
stop = stopwords.words('english')
data['tweet'] = data['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

x = data['tweet'][:len(train)]
y = target

x_train, x_validation_and_test , y_train , y_validation_and_test = train_test_split(x, y,test_size=.3,random_state= SEED)

x_validation , x_test , y_validation , y_test = train_test_split(x_validation_and_test,y_validation_and_test,test_size=0.5,random_state=SEED)


print("Train set has total {0} entries with {1:.2f} % positive, {2:.2f}% negative".format(len(x_train),(len(x_train[y_train==0])/(len(x_train)*1.))*100,
                                                                                          (len(x_train[y_train == 1]) / (len(x_train) * 1.)) * 100))



print("Validation set has total {0} entries with {1:.2f} % positive, {2:.2f}% negative".format(len(x_validation),(len(x_validation[y_validation==0])/(len(x_validation)*1.))*100,
                                                                                          (len(x_validation[y_validation == 1]) / (len(x_validation) * 1.)) * 100))




print("Test set has total {0} entries with {1:.2f} % positive, {2:.2f}% negative".format(len(x_test),(len(x_test[y_test==0])/(len(x_test)*1.))*100,
                                                                                          (len(x_test[y_test == 1]) / (len(x_test) * 1.)) * 100))





def labelize_tweets_ug(tweets,label):

    result = []
    prefix = label
    for i , t in zip(tweets.index,tweets):
        result.append(TaggedDocument(t.split(),[prefix + '_%s' % i ]))

    return result



all_x = pd.concat([x_train,x_validation,x_test])
all_x_w2v = labelize_tweets_ug(all_x,'all')


/Users/subir/anaconda3/envs/DeepLearning/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/subir/anaconda3/envs/DeepLearning/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/subir/anaconda3/envs/DeepLearning/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Text Preprocessing ====>
Train set has total 22373 entries with 92.93 % positive, 7.07% negative
Validation set has total 4794 entries with 92.89 % positive, 7.11% negative
Test set has total 4795 entries with 93.33 % positive, 6.67% negative


In [8]:
# Distributed Bag of Words
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import scale
cores = multiprocessing.cpu_count()
 
model_ug_cbow = Word2Vec(sg=0,size=100,negative=5,window=2,min_count=2,workers=cores,alpha=0.1,min_alpha=0.1)
model_ug_cbow.build_vocab([x.words for x in tqdm(all_x_w2v)])


model_ug_sg = Word2Vec(sg=1,size=100,negative=5,window=2,min_count=2,workers=cores,alpha=0.1,min_alpha=0.1)
model_ug_sg.build_vocab([x.words for x in tqdm(all_x_w2v)])

def get_w2v_mean(tweet, size):
    vec = np.zeros(size).reshape((1,size))
    count = 0
    for word in tweet.split():
        try:
            vec += np.append(model_ug_cbow[word],model_ug_sg[word]).reshape((1,size))
            count += 1
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec

def get_w2v_sum(tweet, size):
    vec = np.zeros(size).reshape((1,size))
    for word in tweet.split():
        try:
            vec += np.append(model_ug_cbow[word],model_ug_sg[word]).reshape((1,size))
        except KeyError:
            continue
    return vec

for epoch in range(30):
    model_ug_cbow.train(utils.shuffle([x.words for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v),epochs=1)
    model_ug_cbow.alpha -=0.002
    model_ug_cbow.min_alpha = model_ug_cbow.alpha

train_vecs_cbow_sg_mean = scale(np.concatenate([get_w2v_mean(z,200) for z in x_train]))
train_vecs_cbow_sg_sum = scale(np.concatenate([get_w2v_sum(z,200) for z in x_train]))

val_vecs_cbow_sg_mean = scale(np.concatenate([get_w2v_mean(z,200) for z in x_validation]))
val_vecs_cbow_sg_sum = scale(np.concatenate([get_w2v_sum(z,200) for z in x_validation]))

clf = LogisticRegression()
clf.fit(train_vecs_cbow_sg_mean, y_train)
print('score',clf.score(val_vecs_cbow_sg_mean,y_validation))


clf = LogisticRegression()
clf.fit(train_vecs_cbow_sg_sum, y_train)
print('score',clf.score(val_vecs_cbow_sg_sum,y_validation))

100%|██████████| 31962/31962 [00:00<00:00, 2078133.97it/s]


score 0.9468085106382979
score 0.9518147684605757


In [13]:
embedding_index = {}

for w in model_ug_cbow.wv.vocab.keys():
    embedding_index[w] = np.append(model_ug_cbow.wv[w], model_ug_sg.wv[w])

In [14]:
embedding_index.get('facebook')

array([-9.4101411e-01,  7.0059037e-01,  8.3010459e-01, -7.6240128e-01,
        9.2581913e-02, -3.9489813e-02, -3.3957180e-01, -1.9511002e-01,
        1.3541147e+00, -1.7232928e-01, -3.2292948e+00, -6.1934370e-01,
       -6.1544079e-01,  7.9256278e-01, -6.4990640e-01,  1.7944692e+00,
        4.5546997e-01,  7.7703619e-01,  2.5435334e-01, -6.3907212e-01,
        2.2822945e+00, -3.1438178e-01, -1.7934468e+00,  7.9437464e-01,
       -1.0218585e+00,  1.7109734e+00,  7.7585500e-01,  1.5414184e-01,
        7.4565250e-01,  1.5159445e+00,  1.2781268e+00,  1.4685680e+00,
       -2.4625416e-01, -6.6322461e-03, -9.9509485e-02, -9.6901143e-01,
        2.2899323e+00,  8.9628136e-01,  1.6564355e+00,  5.7904100e-01,
        1.2763214e+00,  1.1964790e+00,  5.3964864e-02, -5.9881747e-01,
       -3.3820705e+00, -3.0595270e-01,  4.3726271e-01,  6.4467233e-01,
        1.5943125e+00,  1.0897419e+00,  8.7324619e-01, -5.0665039e-01,
       -3.1724146e-01, -7.9429191e-01,  1.1440791e+00, -5.1362610e-01,
      

In [9]:
# Distributed Memory Concatenation

model_ug_dmc = Doc2Vec(dm=1,dm_concat=1,vector_size=100,negative=5,min_count=2,workers=cores,alpha=0.1,min_alpha=0.1)
model_ug_dmc.build_vocab([x for x in tqdm(all_x_w2v)])


for epoch in range(30):
    model_ug_dmc.train(utils.shuffle([x for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v),epochs=1)
    model_ug_dmc.alpha -=0.002
    model_ug_dmc.min_alpha = model_ug_dmc.alpha

train_vecs_dmc = get_vectors(model_ug_dmc, x_train, 100)
validation_vecs_dmc = get_vectors(model_ug_dmc, x_validation , 100)


clf = LogisticRegression()
clf.fit(train_vecs_dmc, y_train)
print('score',clf.score(validation_vecs_dmc,y_validation))

100%|██████████| 31962/31962 [00:00<00:00, 2678220.85it/s]


score 0.928869420108469


In [10]:
# Distributed Memory Mean

model_ug_dmm = Doc2Vec(dm=1,dm_mean=1,vector_size=100,negative=5,min_count=2,workers=cores,alpha=0.1,min_alpha=0.1)
model_ug_dmm.build_vocab([x for x in tqdm(all_x_w2v)])


for epoch in range(30):
    model_ug_dmm.train(utils.shuffle([x for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v),epochs=1)
    model_ug_dmm.alpha -=0.002
    model_ug_dmm.min_alpha = model_ug_dmm.alpha

train_vecs_dmm = get_vectors(model_ug_dmm, x_train, 100)
validation_vecs_dmm = get_vectors(model_ug_dmm, x_validation , 100)


clf = LogisticRegression()
clf.fit(train_vecs_dmm, y_train)
print('score',clf.score(validation_vecs_dmm,y_validation))

100%|██████████| 31962/31962 [00:00<00:00, 2914945.52it/s]


score 0.9309553608677513


In [11]:
def get_concat_vectors(model1,model2, corpus, size):
    vecs = np.zeros((len(corpus), size))
    n = 0
    for i in corpus.index:
        prefix = 'all_' + str(i)
        vecs[n] = np.append(model1.docvecs[prefix],model2.docvecs[prefix])
        n += 1
    return vecs

train_vecs_dbow_dmc = get_concat_vectors(model_ug_dbow,model_ug_dmc, x_train, 200)
validation_vecs_dbow_dmc = get_concat_vectors(model_ug_dbow,model_ug_dmc, x_validation, 200)

clf = LogisticRegression()
clf.fit(train_vecs_dbow_dmc, y_train)
clf.score(validation_vecs_dbow_dmc, y_validation)

0.9413850646641635

In [12]:
train_vecs_dbow_dmm = get_concat_vectors(model_ug_dbow,model_ug_dmm, x_train, 200)
validation_vecs_dbow_dmm = get_concat_vectors(model_ug_dbow,model_ug_dmm, x_validation, 200)

clf = LogisticRegression()
clf.fit(train_vecs_dbow_dmm, y_train)
clf.score(validation_vecs_dbow_dmm, y_validation)

0.9397163120567376

In [13]:
train_vecs_dmm_dmc = get_concat_vectors(model_ug_dmm,model_ug_dmc, x_train, 200)
validation_vecs_dmm_dmc = get_concat_vectors(model_ug_dmm,model_ug_dmc, x_validation, 200)

clf = LogisticRegression()
clf.fit(train_vecs_dmm_dmc, y_train)
clf.score(validation_vecs_dmm_dmc, y_validation)

0.9309553608677513